In [16]:
import numpy as np
import gurobipy as gp
from Demos.win32netdemo import verbose_level
from gurobipy import GRB

In [17]:
A = [0, 1]

P_wait = np.array([
    [0.97, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.002, 0.002, 0, 0.001],
    [0.015, 0.965, 0.005, 0.005, 0.0025, 0.0012, 0.0012, 0.0025, 0.0013, 0.0013],
    [0.005, 0.005, 0.96, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.0025, 0.0025],
    [0, 0.005, 0.005, 0.95, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005],
    [0, 0, 0.005, 0.005, 0.94, 0.02, 0.01, 0.01, 0.004, 0.006],
    [0, 0, 0, 0.005, 0.005, 0.93, 0.02, 0.02, 0.013, 0.007],
    [0, 0, 0, 0, 0.005, 0.007, 0.93, 0.03, 0.02, 0.008],
    [0, 0, 0, 0, 0, 0.005, 0.001, 0.914, 0.04, 0.04],
    [0, 0, 0, 0, 0, 0, 0.01, 0.01, 0.9, 0.08],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])

S = list(range(0,11))  # 10 is transplant, 9 is death

rT = dict(zip(list(range(0,11)), [100,96,90,84,76,68,56,40,25,0, 0]))

rW = dict(zip(list(range(0, 11)), [0.99,0.98,0.97,0.96,0.93,0.9,0.87,0.84,0.8,0, 0]))
lam = 0.9998
epsilon = 1e-4

In [18]:
def P(j, s, a):

    if s == 10 and j == 10:  # Transplant Absorbing State
        p = 1
    elif a == 1 and j == 10:  # Transplant (from non-transplant state)
        p = 1
    elif a == 0 and j != 10 and s != 10:
        p = P_wait[s, j]
    else:
        p = 0

    return p

def r(s, a):
    if a == 0:
        rwd = rW[s]
    elif a == 1:
        rwd = rT[s]
    else:
        rwd = 0

    return rwd

In [19]:
m = gp.Model()
v = m.addVars(S)
m.update()

In [20]:
bellman = m.addConstrs((v[s] - (lam * gp.quicksum(P(j, s, a) * v[j] for j in S)) >= r(s, a) for s in S for a in A), name='bellman')
m.update()

In [21]:
m.setObjective(gp.quicksum(v[s] for s in S), GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22 rows, 11 columns and 91 nonzeros
Model fingerprint: 0x11cceacd
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 1e+02]
Presolve removed 16 rows and 4 columns
Presolve time: 0.00s
Presolved: 6 rows, 7 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.2969728e+31   6.195845e+30   7.296973e+01      0s
       2    6.8924892e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.892489198e+02


In [22]:
for var in v:
    print(var, v[var].X)

0 120.97365564288195
1 118.96697599427944
2 100.30828815570685
3 84.0
4 76.0
5 68.0
6 56.0
7 40.0
8 25.0
9 0.0
10 0.0


In [23]:
# DUAL PROBLEM

dual = gp.Model()
x = dual.addVars(S, A, lb=0.0)
dual.update()

In [24]:
dual_cons = dual.addConstrs(gp.quicksum(x[s, a] for a in A) - gp.quicksum(lam * P(s, j, a) * x[j, a] for j in S for a in A) == 1 for s in S)
dual.update()

In [25]:
dual.setObjective(gp.quicksum(r(s, a) * x[s, a] for s in S for a in A), GRB.MAXIMIZE)
dual.update()
dual.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11 rows, 22 columns and 91 nonzeros
Model fingerprint: 0xdce0deb0
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [8e-01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 7 columns
Presolve time: 0.01s
Presolved: 9 rows, 15 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3004000e+32   7.719661e+30   4.300400e+02      0s
       9    6.8924892e+02   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.892489198e+02


In [26]:
x

{(0, 0): <gurobi.Var C0 (value 65.88882841458137)>,
 (0, 1): <gurobi.Var C1 (value 0.0)>,
 (1, 0): <gurobi.Var C2 (value 52.763574250624764)>,
 (1, 1): <gurobi.Var C3 (value 0.0)>,
 (2, 0): <gurobi.Var C4 (value 39.638320086668124)>,
 (2, 1): <gurobi.Var C5 (value 0.0)>,
 (3, 0): <gurobi.Var C6 (value 0.0)>,
 (3, 1): <gurobi.Var C7 (value 1.9894472851498732)>,
 (4, 0): <gurobi.Var C8 (value 0.0)>,
 (4, 1): <gurobi.Var C9 (value 1.4947236425749366)>,
 (5, 0): <gurobi.Var C10 (value 0.0)>,
 (5, 1): <gurobi.Var C11 (value 1.4261447145784296)>,
 (6, 0): <gurobi.Var C12 (value 0.0)>,
 (6, 1): <gurobi.Var C13 (value 1.2941309081973535)>,
 (7, 0): <gurobi.Var C14 (value 0.0)>,
 (7, 1): <gurobi.Var C15 (value 1.3627098361938605)>,
 (8, 0): <gurobi.Var C16 (value 0.0)>,
 (8, 1): <gurobi.Var C17 (value 1.167654909053134)>,
 (9, 0): <gurobi.Var C18 (value 6167.652798510841)>,
 (9, 1): <gurobi.Var C19 (value 0.0)>,
 (10, 0): <gurobi.Var C20 (value 48665.32166744755)>,
 (10, 1): <gurobi.Var C21 (va

In [31]:
d = np.empty(len(S))

for var in x:
    if x[var].X > 0:
        # print(var)
        d[var[0]] = var[1]

print('d = ', d)

d =  [0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
